In [20]:
import numpy as np 
import pandas as pd 

In [21]:
data_dir = "C:/Users/singhy/Desktop/Chicago/cps_data/inflation/raw_data"
output_dir = "C:/Users/singhy/Desktop/Chicago/cps_data/inflation/output"

In [23]:
# Stock of Vacancies 
vacancy = pd.read_excel(f"{data_dir}/barnichon/CompositeHWI.xlsx")

# Stock of Employed and Unemployed Workers 
stocks = pd.read_excel(f"{data_dir}/fred_employment/employment.xls", engine = 'xlrd')

# jolts 
jolts = pd.read_excel(f"{data_dir}/JOLTS/jolts_level.xls", engine='xlrd')

# Consumer Price Index 
cpi = pd.read_excel(f"{data_dir}/CPI/CPIAUCSL.xls", engine='xlrd')

In [27]:
jolts.iloc[13:]

,FRED Graph Observations,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
13,2000-12-01 00:00:00,2882,5088,5426,2018
14,2001-01-01 00:00:00,3245,5234,5722,2220
15,2001-02-01 00:00:00,3053,5097,5303,1855
16,2001-03-01 00:00:00,3054,4762,5528,2133
17,2001-04-01 00:00:00,3163,4615,5204,1883
...,...,...,...,...,...
294,2024-05-01 00:00:00,3403,8230,5655,1678
295,2024-06-01 00:00:00,3214,7910,5248,1560
296,2024-07-01 00:00:00,3243,7711,5416,1713
297,2024-08-01 00:00:00,3178,7861,5435,1668


In [28]:
# Basic Processing of historical vacancies 
vacancy.columns = ['date', 'V', 'V_rate']  
vacancy = vacancy.iloc[8:].reset_index(drop=True)
vacancy = vacancy.dropna(subset=['date', 'V'])
vacancy = vacancy.drop(['V_rate'], axis = 1)
vacancy['V'] = vacancy['V'].astype(float)

# Basic Processing of stocks 
stocks.columns = ['date', 'E', 'U']
stocks = stocks.iloc[11:].reset_index(drop=True)
stocks = stocks.dropna(subset=['date', 'E', 'U'])
stocks['date'] = pd.to_datetime(stocks['date'])
stocks['U'] = stocks['U'].astype(float)


# CPI-U 
cpi = cpi.iloc[11:].reset_index(drop=True)
cpi = cpi.rename(columns={'FRED Graph Observations': 'date', 
                            'Unnamed: 1':               'P'   })

cpi['date'] = pd.to_datetime(cpi['date'])
cpi['P'] = pd.to_numeric(cpi['P'], errors='coerce')
cpi['P_12m_change'] = cpi['P'].pct_change(periods=12) * 100

# JOLTS 
jolts.columns = ['date', 'tot_quits', 'vacancy_stock', 'tot_hires', 'tot_layoffs']
jolts = jolts.iloc[13:].reset_index(drop=True)
jolts['date'] = pd.to_datetime(jolts['date'])


In [6]:
# Define the mapping function with increased tolerance
def map_to_month(decimal_date):
    fraction = decimal_date - int(decimal_date)
    #print("decimal_date", decimal_date)
    #print("integer decimal_date", int(decimal_date))
    #print('fraction', fraction)
    
    # Define exact mappings with slightly higher tolerance
    if np.isclose(fraction, 1, atol=0.02):
        month = 1  # January
    elif np.isclose(fraction, 0.08, atol=0.02):
        month = 2  # February
    elif np.isclose(fraction, 0.17, atol=0.02):
        month = 3  # March
    elif np.isclose(fraction, 0.25, atol=0.02):
        month = 4  # April
    elif np.isclose(fraction, 0.33, atol=0.02):
        month = 5  # May
    elif np.isclose(fraction, 0.42, atol=0.02):
        month = 6  # June
    elif np.isclose(fraction, 0.50, atol=0.02):
        month = 7  # July
    elif np.isclose(fraction, 0.58, atol=0.02):
        month = 8  # August
    elif np.isclose(fraction, 0.67, atol=0.02):
        month = 9  # September
    elif np.isclose(fraction, 0.75, atol=0.02):
        month = 10  # October
    elif np.isclose(fraction, 0.83, atol=0.02):
        month = 11  # November
    elif np.isclose(fraction, 0.92, atol=0.02):
        month = 12  # December
    else:
        raise ValueError(f"Fraction {fraction} does not match any month")
    
    return month

# Define the conversion function
def convert_to_datetime(decimal_date):
    decimal_date = float(decimal_date)
    year = int(decimal_date)
    month = map_to_month(decimal_date)
    return pd.Timestamp(year=year, month=month, day=1)

# Apply the conversion function to your dataset
vacancy['date'] = vacancy['date'].apply(convert_to_datetime)

In [7]:
temp = stocks.merge(cpi, on = ['date'], how = 'inner')

In [8]:
temp2 = temp.merge(jolts, on = ['date'], how='outer')

In [9]:
final = temp2.merge(vacancy, on = ['date'], how = 'outer')

In [10]:
final

,date,E,U,P,P_12m_change,tot_quits,vacancy_stock,tot_hires,tot_layoffs,V
0,1948-01-01,58061,2034.0,23.680,NaN,NaN,NaN,NaN,NaN,NaN
1,1948-02-01,58196,2328.0,23.670,9.481961,NaN,NaN,NaN,NaN,NaN
2,1948-03-01,57671,2399.0,23.500,6.818182,NaN,NaN,NaN,NaN,NaN
3,1948-04-01,58291,2386.0,23.820,8.272727,NaN,NaN,NaN,NaN,NaN
4,1948-05-01,57854,2118.0,24.010,9.384966,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
916,2024-05-01,161083,6649.0,313.225,3.250210,3403.0,8230.0,5655.0,1678.0,NaN
917,2024-06-01,161199,6811.0,313.049,2.975629,3214.0,7910.0,5248.0,1560.0,NaN
918,2024-07-01,161266,7163.0,313.534,2.923566,3277.0,7673.0,5521.0,1762.0,NaN
919,2024-08-01,161434,7115.0,314.121,2.591227,NaN,NaN,NaN,NaN,NaN


In [95]:
final = final[(final['date'] >= '1951-01-01') & (final['date'] <= '2024-06-01')]

In [96]:
final['V'] = final['V'].fillna(final['vacancy_stock'])

In [97]:
final['date'] = pd.to_datetime(final['date'])

In [98]:
final['L'] = final['E'] + final['U']

final['U_rate'] = (final['U'] / final['L']) * 100 
final['V_rate'] = (final['V'] / final['L']) * 100


final['tightness'] = final['V'] / final['U']
final['ln_tightness'] = np.log(final['V']) - np.log(final['U'])

In [99]:
keep = ['date', 'P_12m_change', 'U_rate', 'V_rate', 'tightness', 'ln_tightness']
final = final[keep]

In [100]:
final.reset_index(drop=True)

,date,P_12m_change,U_rate,V_rate,tightness,ln_tightness
0,1951-01-01,7.954062,3.721283,4.118172,1.106654,0.101341
1,1951-02-01,9.402795,3.426786,3.71603,1.084407,0.081033
2,1951-03-01,9.475465,3.398586,4.085784,1.202201,0.184154
3,1951-04-01,9.598309,3.104776,4.048809,1.304058,0.265481
4,1951-05-01,9.339504,2.991425,4.085784,1.365832,0.311764
...,...,...,...,...,...,...
877,2024-02-01,3.165743,3.857226,5.263818,1.364664,0.310908
878,2024-03-01,3.475131,3.829179,4.976324,1.299580,0.262041
879,2024-04-01,3.357731,3.864677,4.714168,1.219809,0.198694
880,2024-05-01,3.250210,3.964062,4.906637,1.237780,0.213320


In [101]:
final.to_csv(f"{output_dir}/data/historical_data.csv", index=False)